In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from scipy.spatial.distance import directed_hausdorff

# CLASS NAMES
CLASS_NAMES = [
    "Enlarged Cardiomediastinum", 
    "Cardiomegaly", 
    "Airspace Opacity",
    "Lung Lesion", 
    "Edema", 
    "Consolidation", 
    "Atelectasis",
    "Pneumothorax", 
    "Pleural Effusion", 
    "Support Devices"
]

# DIR PATHS
GT_ROOT   = "CheXpert/infection masks/infection_masks_test"
PRED_ROOT = "final_predicted_lime_masks"


# ============================================================
# METRIC FUNCTIONS
# ============================================================
def iou_score(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred).sum()
    union        = np.logical_or(y_true, y_pred).sum()
    return intersection / union if union > 0 else 1.0

def dice_score(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred).sum()
    denom = y_true.sum() + y_pred.sum()
    return 2 * intersection / denom if denom > 0 else 1.0

def hausdorff_distance(y_true, y_pred):
    pts_true = np.column_stack(np.where(y_true))
    pts_pred = np.column_stack(np.where(y_pred))
    if len(pts_true) == 0 or len(pts_pred) == 0:
        return np.nan
    return max(
        directed_hausdorff(pts_true, pts_pred)[0],
        directed_hausdorff(pts_pred, pts_true)[0]
    )


In [2]:
metrics = {
    cls: {
        "IoU": [], "Dice": [], "Hausdorff": [],
        "F1": [], "Accuracy": []
    }
    for cls in CLASS_NAMES
}


In [3]:
patients = sorted(os.listdir(PRED_ROOT))

for patient in tqdm(patients, desc="Patients"):

    pred_patient_dir = os.path.join(PRED_ROOT, patient)
    if not os.path.isdir(pred_patient_dir):
        continue  # skip .DS_Store or other non-folder files

    gt_patient_dir = os.path.join(GT_ROOT, patient)
    if not os.path.isdir(gt_patient_dir):
        continue

    for study in os.listdir(pred_patient_dir):

        pred_study_dir = os.path.join(pred_patient_dir, study)
        if not os.path.isdir(pred_study_dir):
            continue  # skip .DS_Store inside patient folder

        gt_study_dir = os.path.join(gt_patient_dir, study)
        if not os.path.isdir(gt_study_dir):
            continue

        for file in os.listdir(pred_study_dir):

            pred_path = os.path.join(pred_study_dir, file)
            if not os.path.isfile(pred_path):
                continue  # skip .DS_Store inside study folder

            if not file.endswith(".png"):
                continue

            gt_path = os.path.join(gt_study_dir, file)
            if not os.path.exists(gt_path):
                continue

            # ---- Load both masks ----
            pred = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
            gt   = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

            if pred is None or gt is None:
                continue

            pred_bin = (pred > 0).astype(np.uint8)
            gt_bin   = (gt > 0).astype(np.uint8)

            # ---- Skip blank-vs-blank ----
            if pred_bin.sum() == 0 and gt_bin.sum() == 0:
                continue

            # ---- Determine class name from filename ----
            cls_name = None
            for cls in CLASS_NAMES:
                if cls in file:
                    cls_name = cls
                    break

            if cls_name is None:
                continue  # filename doesn't match any class

            # ---- Compute metrics ----
            metrics[cls_name]["IoU"].append(iou_score(gt_bin, pred_bin))
            metrics[cls_name]["Dice"].append(dice_score(gt_bin, pred_bin))
            metrics[cls_name]["Hausdorff"].append(hausdorff_distance(gt_bin, pred_bin))
            metrics[cls_name]["F1"].append(f1_score(gt_bin.flatten(), pred_bin.flatten(), zero_division=1))
            metrics[cls_name]["Accuracy"].append(accuracy_score(gt_bin.flatten(), pred_bin.flatten()))


Patients: 100%|██████████| 501/501 [00:18<00:00, 26.98it/s]


In [5]:
def safe_mean(arr):
    return np.nan if len(arr) == 0 else np.nanmean(arr)

print("\n============ PER-CLASS METRICS (Filtered) ============")

for cls in CLASS_NAMES:
    print(f"\n--- {cls} ---")
    print(f"IoU       : {safe_mean(metrics[cls]['IoU']):.4f}")
    print(f"Dice      : {safe_mean(metrics[cls]['Dice']):.4f}")
    print(f"Hausdorff : {safe_mean(metrics[cls]['Hausdorff']):.2f}")
    print(f"F1 Score  : {safe_mean(metrics[cls]['F1']):.4f}")
    print(f"Accuracy  : {safe_mean(metrics[cls]['Accuracy']):.4f}")



============ PER-CLASS METRICS (Filtered) ============

--- Airspace Opacity ---
IoU       : 0.0000
Dice      : 0.0000
Hausdorff : nan
F1 Score  : 0.0000
Accuracy  : 0.9129

--- Atelectasis ---
IoU       : 0.0045
Dice      : 0.0077
Hausdorff : 113.50
F1 Score  : 0.0077
Accuracy  : 0.9119

--- Cardiomegaly ---
IoU       : 0.0047
Dice      : 0.0082
Hausdorff : 128.89
F1 Score  : 0.0082
Accuracy  : 0.8434

--- Consolidation ---
IoU       : 0.0000
Dice      : 0.0000
Hausdorff : nan
F1 Score  : 0.0000
Accuracy  : 0.9060

--- Edema ---
IoU       : 0.0676
Dice      : 0.1062
Hausdorff : 81.55
F1 Score  : 0.1062
Accuracy  : 0.8430

--- Enlarged Cardiomediastinum ---
IoU       : 0.0000
Dice      : 0.0000
Hausdorff : nan
F1 Score  : 0.0000
Accuracy  : 0.8247

--- Lung Lesion ---
IoU       : 0.0000
Dice      : 0.0000
Hausdorff : nan
F1 Score  : 0.0000
Accuracy  : 0.9738

--- Pleural Effusion ---
IoU       : 0.0005
Dice      : 0.0010
Hausdorff : 102.12
F1 Score  : 0.0010
Accuracy  : 0.9283

--- Pn

/var/folders/fz/sd4mlm5j5xq82d0vlcyb4sgm0000gn/T/ipykernel_3307/3292207576.py:2: RuntimeWarning: Mean of empty slice
  return np.nan if len(arr) == 0 else np.nanmean(arr)
